## Setting up AML WS and compute

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: mneu-ml-dev-wbamlwst-001
Azure region: northeurope
Subscription id: 3430db00-3faf-4f9d-9eac-80a6c22fdac4
Resource group: mneu-rg-dev-workbenchamlws-001


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "compute-cluster-northeurope"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

kianminlim-GPU ComputeInstance Succeeded
CPU-cluster-001 AmlCompute Succeeded
compute-cluster-northeurope AmlCompute Succeeded
angeladeloureiro-GPU ComputeInstance Succeeded
tanzelleoberholster-GPU ComputeInstance Succeeded


## Registering the data

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = ["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"]
dataset = TabularDatasetFactory.from_delimited_files(path=data_path)

In [5]:
from azureml.core.dataset import Dataset
from azureml.exceptions import UserErrorException
dataset_registered = False
try:
    temp = Dataset.get_by_name(workspace = ws, name = 'udacity-bankmarketing')
    dataset_registered = True
except UserErrorException:
    print("The dataset bank marketing data is not registered in workspace yet.")

if not dataset_registered:
    print('registering the data')
    dataset = dataset.register(workspace = ws,
                               name = 'udacity-bankmarketing',
                               description='udacity bank marketing training and test dataset',
                               create_new_version=True)

In [8]:
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [9]:
type(ds)

azureml.data.tabular_dataset.TabularDataset

Bad pipe message: %s [b'J\xaa\x0eU\xd2\x0bo V\xb5d%\xe0\x80\xa0f\x04\xd4 \xdbm?\x01\x111\x11\x91\x8c\xefUd\x9d\xbfHP\x12\x1b\x05\xf0\xa5N\x19\xda\xad\xba\xa3\xd3\x1d:j\xed\x00\x08\x13\x02\x13\x03', b'\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127', b'.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00']
Bad pipe message: %s [b'\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\xf9e\xfb\xda\x9d(\xfa\xf3\x1e\x9dI\x19AQW:l\x13 \xed', b'\x95\xf0\x97\xee\x89\x13U\xa7\x99Y\xf9\xe9\xc9\x83\x89N\x04\xdd\x9a\x89hb^e\xd3\xdf~\xb5.\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t']
Bad pipe message: %s [b"\xf5Xx\xc4q\xf3\xe2cx\nIG\xbf\xb7\x0c)D\r\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\

In [6]:
dataset = Dataset.get_by_name(ws, 'udacity-bankmarketing')


In [7]:
type(dataset)

azureml.data.tabular_dataset.TabularDataset

In [ ]:
from azureml.core import Environment, ScriptRunConfig
import os

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = './training'


# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

args = ['--data-folder', dataset.as_named_input('mnist').as_mount()]  ## define data mounting


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=sklearn_env)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': loguniform(-6, -1)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)







# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

The dataset mnist-dataset is not registered in workspace yet.


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###